*This Python script automates the extraction, cleaning, and consolidation of the extensive Freddie Mac Single-Family Loan-Level Dataset, a standard and widely-used resource in mortgage-backed securities analysis. Designed for computational efficiency, the code processes a decade of quarterly data (2014-2024) by systematically reading origination and performance files.Through a robust process of deduplication and merging, the script constructs a reliable panel dataset that serves as a critical input for forecasting loan performance and measuring portfolio risk.*

### **Origination Data**

#### Within a Quarter:
The origination file for each quarter contains unique Loan Sequence Number values specific to that quarter, as it records only new loans originated during that period.

#### Across Quarters:
Origination data across quarters (e.g., Q1, Q2, Q3, Q4) should generally be unique per loan, with each Loan Sequence Number assigned at origination and not reused.

However, a loan might appear in multiple quarters’ origination files if:
 - Its details were revised (e.g., due to corrections or amendments).
 - Historical data is re-included in later quarters for completeness, though this is rare.

### **Performance Data**

#### Within a Quarter:
- The performance file for a given quarter includes multiple entries for the same Loan Sequence Number, representing monthly performance updates.
- For example, a loan might have records for Monthly Reporting Period values like 202401 (January), 202402 (February), and 202403 (March) in Q1 2024.

#### Across Quarters:
- Performance data extends across multiple quarters, with a loan’s performance tracked in each quarter’s file up to the latest available data.
- Performance data for a loan is tracked over time, but Freddie Mac provides it in a single performance file per origination quarter, not separate files for each reporting quarter.
- For example, a loan originated in Q1 2024 will have all its performance records (e.g., January, February, March 2024, and continuing through the latest available data, such as March 2025) consolidated in the performance file tied to Q1 2024.
- This means there are not multiple records per loan ID across separate quarterly performance files (e.g., one for Q2, Q3, Q4). Instead, the single performance file for the origination quarter contains all monthly updates up to the most recent data cutoff, avoiding duplication across quarters.

### **Relationship Between Origination and Performance Data**

#### Loan IDs in Performance vs. Origination: 
- The performance data for a specific quarter does not include Loan Sequence Numbers beyond those originated in its corresponding origination quarter.
- Instead, each performance file (e.g., performance_2024Q2) is tied to the loans originated in that same quarter (e.g., origination_2024Q2) and tracks their performance from origination through the latest available data.
- This means performance files are cohort-specific and do not aggregate loans from previous quarters.

#### Merge Behavior
- An inner merge (pd.merge(..., how='inner')) will match only the loans present in both the origination and performance files for that quarter, ensuring consistency.

### Data Extraction Code Approach

- Process data quarterly across all years (2014-2024).
- For origination data, keep the first instance of each Loan Sequence Number globally across all years and quarters.
- For performance data, keep the last instance of each Loan Sequence Number globally across all years and quarters.
- Use an inner join to merge origination and performance data.
- Ensure computational efficiency given the large dataset size and quarterly separation.

In [ ]:
import pandas as pd
import numpy as np
import os
import re
from pathlib import Path

In [ ]:
# Directory containing the data folders and Excel layout file
data_dir = '/Users/dr/Documents/GitHub/MBS_RiskManagement/data/'
layout_file = '/Users/dr/Documents/GitHub/MBS_RiskManagement/READ_ME/SF LLD File Layout Release 44.xlsx'

In [ ]:
# Function to parse sheet content from Excel
def parse_sheet_from_excel(sheet_name):
    df = pd.read_excel(layout_file, sheet_name=sheet_name, header=None)
    columns = []
    dtypes = {}
    start_row = df.index[df[0].str.contains('FIELD POSITION', na=False)].tolist()
    if start_row:
        start_row = start_row[0] + 1
    else:
        start_row = 0
    
    for index, row in df.iloc[start_row:].iterrows():
        if pd.isna(row[0]) or not isinstance(row[1], str):
            break
        attribute_name = row[1].strip()
        data_type = row[2].strip() if pd.notna(row[2]) else 'object'
        columns.append(attribute_name)
        if 'Alpha' in data_type or 'Alpha Numeric' in data_type or '- PYYQnXXXXXXX' in data_type:
            dtypes[attribute_name] = 'object'
        elif 'Numeric' in data_type and not ' - ' in data_type:
            dtypes[attribute_name] = 'Int64'
        elif 'Numeric - ' in data_type:
            dtypes[attribute_name] = 'float64'
        elif 'Date' in data_type:
            dtypes[attribute_name] = 'datetime64[ns]'
        else:
            dtypes[attribute_name] = 'object'
    print(f"{sheet_name} columns: {columns}")
    print(f"{sheet_name} dtypes: {dtypes}")
    return columns, dtypes

In [ ]:
# Parse Origination and Performance sheets from Excel
origination_columns, origination_dtypes = parse_sheet_from_excel('Origination Data File')
performance_columns, performance_dtypes = parse_sheet_from_excel('Monthly Performance Data File')

In [ ]:
# Create the extracted_data folder 
extracted_data_dir = os.path.join('/Users/dr/Documents/GitHub/MBS_RiskManagement/', 'extracted_data') 
os.makedirs(extracted_data_dir, exist_ok=True)

In [ ]:
from datetime import datetime

def parse_yyyymm(val):
    """Convert YYYYMM or YYYYMMDD strings into datetime."""
    if pd.isna(val):
        return pd.NaT
    val = str(val).strip()
    if len(val) == 6:  # YYYYMM
        return datetime.strptime(val, "%Y%m")
    elif len(val) == 8:  # YYYYMMDD
        return datetime.strptime(val, "%Y%m%d")
    else:
        return pd.to_datetime(val, errors="coerce")  # fallback


In [ ]:
# Years and quarters to process 
years = range(2014, 2025) # 2014 to 2024 inclusive 
quarters = ['Q1', 'Q2', 'Q3', 'Q4']

In [ ]:
# Function to check available disk space 
def check_disk_space(path, required_mb=100): 
    stat = os.statvfs(path) 
    free_mb = (stat.f_bavail * stat.f_frsize) / (1024 * 1024) 
    if free_mb < required_mb: 
        print(f"Warning: Only {free_mb:.1f} MB free on {path}. Need at least {required_mb} MB. Free up space or adjust output.") 
        return False 
    return True

In [ ]:
# Global dictionaries to track first origination and last performance 
origination_first = {} 
performance_last = {}

In [ ]:
# Process each year and quarter
for year in years:
    for quarter in quarters:
        folder_path = os.path.join(data_dir, f'historical_data_{year}{quarter}')
        
        if os.path.exists(folder_path) and os.path.isdir(folder_path):
            print(f"Processing folder: {folder_path}")
            
            # Check disk space before processing
            if not check_disk_space(extracted_data_dir, required_mb=500):
                break
            
            # ------------------------
            # Process origination data
            # ------------------------
            origination_file_pattern = f'historical_data_{year}{quarter}.txt'
            origination_file = os.path.join(folder_path, origination_file_pattern)

            if os.path.exists(origination_file):
                chunk_iterator = pd.read_csv(
                    origination_file,
                    sep='|',
                    header=None,
                    names=origination_columns,
                    encoding='utf-8',
                    dtype={col: 'object' for col in origination_columns},  # read everything as string first
                    chunksize=100000,
                    low_memory=False
                )

                for chunk in chunk_iterator:
                    # Parse origination date columns
                    for col in ['First Payment Date', 'Maturity Date']:
                        if col in chunk.columns:
                            chunk[col] = chunk[col].apply(parse_yyyymm)

                    # Apply other dtypes
                    for col, dtype in {k: v for k, v in origination_dtypes.items()
                                       if k not in ['First Payment Date', 'Maturity Date']}.items():
                        if col in chunk.columns:
                            chunk[col] = chunk[col].astype(dtype, errors='ignore')

                    # Keep FIRST occurrence per Loan ID (global)
                    chunk = chunk.sort_values(['Loan Sequence Number', 'First Payment Date'])
                    first_rows = chunk.groupby('Loan Sequence Number').head(1)
                    for loan_id, row in first_rows.set_index('Loan Sequence Number').to_dict('index').items():
                        if loan_id not in origination_first:
                            origination_first[loan_id] = row
            else:
                print(f"Origination file not found: {origination_file_pattern}")

            
            # ------------------------
            # Process performance data
            # ------------------------
            performance_file_pattern = f'historical_data_time_{year}{quarter}.txt'
            performance_file = os.path.join(folder_path, performance_file_pattern)

            if os.path.exists(performance_file):
                chunk_iterator = pd.read_csv(
                    performance_file,
                    sep='|',
                    header=None,
                    names=performance_columns,
                    encoding='utf-8',
                    dtype={col: 'object' for col in performance_columns},  # read everything as string first
                    chunksize=100000,
                    low_memory=False
                )
        
                for chunk in chunk_iterator:
                    # Parse performance date columns
                    for col in ['Monthly Reporting Period', 'Defect Settlement Date', 
                                'Zero Balance Effective Date', 'Due Date of Last Paid Installment (DDLPI)']:
                        if col in chunk.columns:
                            chunk[col] = chunk[col].apply(parse_yyyymm)

                    # Apply other dtypes
                    for col, dtype in {k: v for k, v in performance_dtypes.items()
                                       if col not in ['Monthly Reporting Period', 'Defect Settlement Date',
                                                      'Zero Balance Effective Date', 'Due Date of Last Paid Installment (DDLPI)']}.items():
                        if col in chunk.columns:
                            chunk[col] = chunk[col].astype(dtype, errors='ignore')

                    # Keep LAST occurrence per Loan ID (global)
                    chunk = chunk.sort_values(['Loan Sequence Number', 'Monthly Reporting Period'])
                    last_rows = chunk.groupby('Loan Sequence Number').tail(1)
                    performance_last.update(last_rows.set_index('Loan Sequence Number').to_dict('index'))
            else:
                print(f"Performance file not found: {performance_file_pattern}")


In [ ]:
# Convert global dictionaries to DataFrames
df_origination = pd.DataFrame.from_dict(origination_first, orient='index')
df_performance = pd.DataFrame.from_dict(performance_last, orient='index')

In [ ]:
# Inner join on Loan Sequence Number
merged = df_origination.merge(df_performance, 
                              left_index=True, 
                              right_index=True, 
                              how='inner', 
                              suffixes=('_orig', '_perf'))


In [ ]:

# Save final merged dataset
output_file = os.path.join(extracted_data_dir, "merged_loans_2014_2024.csv")
merged.to_csv(output_file, index=True)

print(f"Final merged dataset written to: {output_file}")
print(f"Total loans in merged set: {len(merged)}")